In [1]:
%load_ext autoreload
%autoreload 2

import checklist
import spacy
import itertools

import checklist.editor
import checklist.text_generation
from checklist.mft import MFT
from checklist.inv_dir import INV, DIR
from checklist.expect import Expect
import numpy as np
import spacy
from checklist.perturb import Perturb

In [2]:
editor = checklist.editor.Editor()
nlp = spacy.load('en_core_web_sm')

In [3]:
import sys
import os
import pandas as pd
pd.set_option('display.max_colwidth', 180)

mltest_path = os.path.abspath(os.path.expanduser("~/sourcetree/ml-tests"))
sys.path.append(mltest_path)
mlfeedback_path = os.path.abspath(os.path.expanduser("~/sourcetree/ml_feedback"))
sys.path.append(mlfeedback_path)

from ml_feedback.analysis.sentiment_utils import *
# get the model
model_names = {
    "orig": "imdb-iclr_orig"
}
from ml_feedback.analysis.sentiment_utils import *
models = load_models_by_name(model_names)

from checklist.pred_wrapper import PredictorWrapper
sentiment = models["orig"]
wrapped_pp = PredictorWrapper.wrap_softmax(sentiment.predict_proba)

/home/wtshuang/datasets/models/imdb-iclr_orig
Namespace(adam_epsilon=1e-08, cache_dir='', config_name='', data_dir='/home/wtshuang/datasets/raw/glue_data//imdb/iclr_orig', device=device(type='cuda'), do_eval=True, do_lower_case=True, do_train=True, eval_all_checkpoints=False, evaluate_during_training=False, fp16=False, fp16_opt_level='O1', gradient_accumulation_steps=1, label_list='None', learning_rate=5e-05, local_rank=-1, logging_steps=50, max_grad_norm=1.0, max_seq_length=512, max_steps=-1, model_name_or_path='bert-base-uncased', model_type='bert', n_gpu=2, no_cuda=False, num_train_epochs=10.0, output_dir='//home/wtshuang/datasets/models//imdb-iclr_orig/', output_mode='classification', overwrite_cache=True, overwrite_output_dir=True, per_gpu_eval_batch_size=8, per_gpu_train_batch_size=8, save_steps=1000000000000, seed=42, server_ip='', server_port='', task_name='sst-2', tokenizer_name='', train_batch_size=16, warmup_steps=0, weight_decay=0.0)


In [4]:
from ml_feedback.analysis.sentiment_utils import *

df = read_one_dataset("semeval_iclr_nonbinary")
if len(df) > 2000:
    df = df.sample(2000)
nlp = spacy.load('en_core_web_sm')
sentences = df.sentence.tolist()
parsed_data = list(nlp.pipe(sentences))
labels = df.label.tolist()
parsed_data[0]

TMZ reports that Seth Rollins caught a ride to a local restaurant in an unmarked cop car Saturday evening when he... http://t.co/s3deESbZCh

In [5]:
def new_pp(data):
    margin_neutral = 1/3.
    mn = margin_neutral / 2.
    pr = wrapped_pp(data)[1][:, 1]
    pp = np.zeros((pr.shape[0], 3))
    neg = pr < 0.5 - mn
    pp[neg, 0] = 1 - pr[neg]
    pp[neg, 2] = pr[neg]
    pos = pr > 0.5 + mn
    pp[pos, 0] = 1 - pr[pos]
    pp[pos, 2] = pr[pos]
    neutral_pos = (pr >= 0.5) * (pr < 0.5 + mn)
    pp[neutral_pos, 1] = 1 - (1 / margin_neutral) * np.abs(pr[neutral_pos] - 0.5)
    pp[neutral_pos, 2] = 1 - pp[neutral_pos, 1]
    neutral_neg = (pr < 0.5) * (pr > 0.5 - mn)
    pp[neutral_neg, 1] = 1 - (1 / margin_neutral) * np.abs(pr[neutral_neg] - 0.5)
    pp[neutral_neg, 0] = 1 - pp[neutral_neg, 1]
    preds = np.argmax(pp, axis=1)
    return preds, pp
    

# MFT

In [6]:
air_noun = ['flight', 'seat', 'pilot', 'staff', 'service', 'customer service', 'aircraft', 'plane', 'food', 'cabin crew', 'company', 'airline', 'crew']
editor.add_lexicon('air_noun', air_noun)
#print(', '.join(editor.suggest('It was {a:bert} {air_noun}.')[:40]))

pos_adj = ['good', 'great', 'excellent', 'amazing', 'extraordinary', 'beautiful', 'fantastic', 'nice', 'incredible', 'exceptional', 'awesome', 'perfect', 'fun', 'happy', 'adorable', 'brilliant', 'exciting', 'sweet', 'wonderful']
neg_adj = ['awful', 'bad', 'horrible', 'tough', 'weird', 'aggressive', 'rough', 'lousy', 'unhappy', 'average', 'difficult', 'poor', 'sad', 'frustrating', 'hard', 'lame', 'nasty', 'annoying', 'boring', 'creepy', 'dreadful', 'ridiculous', 'terrible', 'ugly', 'unpleasant']
neutral_adj = ['American', 'international',  'commercial', 'British', 'private', 'Italian', 'Indian', 'Australian', 'Israeli', ]
editor.add_lexicon('pos_adj', pos_adj, overwrite=True)
editor.add_lexicon('neg_adj', neg_adj, overwrite=True )
editor.add_lexicon('neutral_adj', neutral_adj, overwrite=True)


pos_verb_present = ['like', 'enjoy', 'appreciate', 'love',  'recommend', 'admire', 'value', 'welcome']
neg_verb_present = ['hate', 'dislike', 'regret',  'abhor', 'dread', 'despise' ]
neutral_verb_present = ['see', 'find']
pos_verb_past = ['liked', 'enjoyed', 'appreciated', 'loved', 'admired', 'valued', 'welcomed']
neg_verb_past = ['hated', 'disliked', 'regretted',  'abhorred', 'dreaded', 'despised']
neutral_verb_past = ['saw', 'found']
editor.add_lexicon('pos_verb_present', pos_verb_present, overwrite=True)
editor.add_lexicon('neg_verb_present', neg_verb_present, overwrite=True)
editor.add_lexicon('neutral_verb_present', neutral_verb_present, overwrite=True)
editor.add_lexicon('pos_verb_past', pos_verb_past, overwrite=True)
editor.add_lexicon('neg_verb_past', neg_verb_past, overwrite=True)
editor.add_lexicon('neutral_verb_past', neutral_verb_past, overwrite=True)
editor.add_lexicon('pos_verb', pos_verb_present+ pos_verb_past, overwrite=True)
editor.add_lexicon('neg_verb', neg_verb_present + neg_verb_past, overwrite=True)
editor.add_lexicon('neutral_verb', neutral_verb_present + neutral_verb_past, overwrite=True)



## individual words

In [7]:
test = MFT(pos_adj + pos_verb_present + pos_verb_past, labels=2)
test.run(new_pp)
test.summary(n=3)

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

Predicting 34 examples


Evaluating: 100%|██████████| 5/5 [00:00<00:00,  7.39it/s]

Test cases:      34
Fails (rate):    3 (8.8%)

Example fails:
1 (0.8) value
----
0 (0.8) like
----
1 (0.8) welcomed
----


In [8]:
test.visual_summary()

TestSummarizer(stats={'nTested': 34, 'nFailed': 3, 'nFiltered': 0}, summarizer={'name': 'TESTNAME_PLACEHOLDER'…

## Words in context

In [9]:
data = editor.template('{it} {air_noun} {be} {pos_adj}.', it=['The', 'This', 'That'], be=['is', 'was'])
data += editor.template('{it} {be} {a:pos_adj} {air_noun}.', it=['It', 'This', 'That'], be=['is', 'was'])
data += editor.template('{i} {pos_verb} {the} {air_noun}.', i=['I', 'We'], the=['this', 'that', 'the'])
labels = [2] * len(data)
data += editor.template('{it} {air_noun} {be} {neg_adj}.', it=['That', 'This', 'The'], be=['is', 'was'])
data += editor.template('{it} {be} {a:neg_adj} {air_noun}.', it=['It', 'This', 'That'], be=['is', 'was'])
data += editor.template('{i} {neg_verb} {the} {air_noun}.', i=['I', 'We'], the=['this', 'that', 'the'])
labels += [0] * (len(data) - len(labels))
test = MFT(data, labels=labels)
# test = MFT(data, labels=2)
test.run(new_pp)
test.summary(n=3)
test.visual_summary()

Predicting 8970 examples


Evaluating: 100%|██████████| 1122/1122 [02:00<00:00,  9.28it/s]


Test cases:      8970
Fails (rate):    262 (2.9%)

Example fails:
2 (0.9) That is an aggressive crew.
----
2 (0.8) It was an average cabin crew.
----
2 (1.0) This is a sad plane.
----


TestSummarizer(stats={'nTested': 8970, 'nFailed': 262, 'nFiltered': 0}, summarizer={'name': 'TESTNAME_PLACEHOL…

## Intensifiers and reducers

In [10]:
intens_adj = ['very', 'really', 'absolutely', 'truly', 'extremely', 'quite', 'incredibly', 'amazingly', 'especially', 'exceptionally', 'unbelievably', 'utterly', 'exceedingly', 'rather', 'totally', 'particularly']
intens_verb = [ 'really', 'absolutely', 'truly', 'extremely',  'especially',  'utterly',  'totally', 'particularly', 'highly', 'definitely', 'certainly', 'genuinely', 'honestly', 'strongly', 'sure', 'sincerely']


In [12]:
monotonic_label = Expect.monotonic(increasing=True, tolerance=0.1)
non_neutral_pred = lambda pred, *args, **kwargs: pred != 1
monotonic_label = Expect.slice_pairwise(monotonic_label, non_neutral_pred)

In [13]:
data = editor.template(['{it} {be} {a:pos_adj} {air_noun}.', '{it} {be} {a:intens} {pos_adj} {air_noun}.'] , intens=intens_adj, it=['It', 'This', 'That'], be=['is', 'was'], nsamples=500)
data += editor.template(['{i} {pos_verb} {the} {air_noun}.', '{i} {intens} {pos_verb} {the} {air_noun}.'], intens=intens_verb, i=['I', 'We'], the=['this', 'that', 'the'], nsamples=500)
data += editor.template(['{it} {be} {a:neg_adj} {air_noun}.', '{it} {be} {a:intens} {neg_adj} {air_noun}.'] , intens=intens_adj, it=['It', 'This', 'That'], be=['is', 'was'], nsamples=500)
data += editor.template(['{i} {neg_verb} {the} {air_noun}.', '{i} {intens} {neg_verb} {the} {air_noun}.'], intens=intens_verb, i=['I', 'We'], the=['this', 'that', 'the'], nsamples=500)
test = DIR(data, monotonic_label)
test.run(new_pp)
test.summary(3)


Predicting 4000 examples


Evaluating: 100%|██████████| 500/500 [00:54<00:00,  9.25it/s]

Test cases:      2000
After filtering: 1995 (99.8%)
Fails (rate):    117 (5.9%)

Example fails:
0 (1.0) We dread that airline.
2 (1.0) We truly dread that airline.

----
0 (1.0) It was a tough company.
0 (0.7) It was a particularly tough company.

----
0 (1.0) It was a sad service.
2 (0.7) It was a very sad service.

----


In [14]:
test.visual_summary()

TestSummarizer(stats={'nTested': 1995, 'nFailed': 117, 'nFiltered': 0}, summarizer={'name': 'TESTNAME_PLACEHOL…

In [15]:
monotonic_label = Expect.monotonic(increasing=True, tolerance=0.1)
non_neutral_pred = lambda pred, *args, **kwargs: pred != 1
monotonic_label = Expect.slice_pairwise(monotonic_label, non_neutral_pred)


reducer_adj = ['somewhat', 'kinda', 'mostly', 'probably', 'generally', 'reasonably', 'a little', 'a bit', 'slightly']
monotonic_label_down = Expect.monotonic(increasing=False, tolerance=0.1)
monotonic_label_down = Expect.slice_pairwise(monotonic_label_down, non_neutral_pred)

data = editor.template(['{it} {air_noun} {be} {pos_adj}.', '{it} {air_noun} {be} {red} {pos_adj}.'] , red=reducer_adj, it=['The', 'This', 'That'], be=['is', 'was'], nsamples=1000)
data += editor.template(['{it} {air_noun} {be} {neg_adj}.', '{it} {air_noun} {be} {red} {neg_adj}.'] , red=reducer_adj, it=['The', 'This', 'That'], be=['is', 'was'], nsamples=1000)
test = DIR(data, monotonic_label_down)
test.run(new_pp)
test.summary(3)


Predicting 4000 examples


Evaluating: 100%|██████████| 500/500 [00:54<00:00,  9.25it/s]

Test cases:      2000
After filtering: 53 (2.6%)
Fails (rate):    15 (28.3%)

Example fails:
2 (0.8) The flight was aggressive.
2 (1.0) The flight was mostly aggressive.

----
2 (0.8) The flight was aggressive.
2 (1.0) The flight was somewhat aggressive.

----
2 (0.7) This plane was average.
2 (1.0) This plane was reasonably average.

----


In [16]:
test.visual_summary()

TestSummarizer(stats={'nTested': 53, 'nFailed': 15, 'nFiltered': 0}, summarizer={'name': 'TESTNAME_PLACEHOLDER…

In [1]:
%load_ext autoreload
%autoreload 2

import checklist
import spacy
import itertools

import checklist.editor
import checklist.text_generation
from checklist.mft import MFT
from checklist.inv_dir import INV, DIR
from checklist.expect import Expect
import numpy as np
import spacy
from checklist.perturb import Perturb

In [2]:
import sys
import os
import pandas as pd
pd.set_option('display.max_colwidth', 180)

mltest_path = os.path.abspath(os.path.expanduser("~/sourcetree/ml-tests"))
sys.path.append(mltest_path)
mlfeedback_path = os.path.abspath(os.path.expanduser("~/sourcetree/ml_feedback"))
sys.path.append(mlfeedback_path)

from ml_feedback.analysis.qqp_utils import *


# get the model
model_names = {
    "qqp": "QQP-all",
}
from ml_feedback.analysis.qqp_utils import *
models = load_models_by_name(model_names)

from checklist.pred_wrapper import PredictorWrapper
model = models["qqp"]
new_pp = PredictorWrapper.wrap_softmax(model.predict_proba)

/home/wtshuang/datasets/models/QQP-all
Namespace(adam_epsilon=1e-08, cache_dir='', config_name='', data_dir='/home/wtshuang/datasets/raw/glue_data//QQP/all', device=device(type='cuda'), do_eval=True, do_lower_case=True, do_train=True, eval_all_checkpoints=False, evaluate_during_training=False, fp16=False, fp16_opt_level='O1', gradient_accumulation_steps=1, label_list='None', learning_rate=2e-05, local_rank=-1, logging_steps=50, max_grad_norm=1.0, max_seq_length=512, max_steps=-1, model_name_or_path='bert-base-uncased', model_type='bert', n_gpu=2, no_cuda=False, num_train_epochs=3.0, output_dir='//home/wtshuang/datasets/models//QQP-all/', output_mode='classification', overwrite_cache=True, overwrite_output_dir=True, per_gpu_eval_batch_size=8, per_gpu_train_batch_size=8, save_steps=1000000000000, seed=42, server_ip='', server_port='', task_name='qqp', tokenizer_name='', train_batch_size=16, warmup_steps=0, weight_decay=0.0)


In [3]:
editor = checklist.editor.Editor()
nlp = spacy.load('en_core_web_sm')

In [4]:
# getting the data

df = read_one_dataset("quora", max_size=2000)
parsed_qs = list(zip(
    nlp.pipe(df.question1.tolist()), 
    nlp.pipe(df.question2.tolist())
))
labels = df.is_duplicate.tolist()
parsed_qs[0]

(What 's the best non touchscreen phone under 10K in India ?,
 What is currently the best phone under 10k in India ?)

In [5]:
def change_both_wrapper(fn):
    def change_both(qs):
        q1, q2 = qs
        seed = np.random.randint(100)
        c1 = fn(q1, seed=seed, meta=True)
        c2 = fn(q2, seed=seed, meta=True)
        if not c1 or not c2:
            return
        c1, m1 = c1
        c2, m2 = c2
        return [(q1, q2) for q1, q2, m1, m2 in zip(c1, c2, m1, m2) if m1 == m2]
    return change_both
    
data = Perturb.perturb(parsed_qs, change_both_wrapper(Perturb.change_names), nsamples=500)
test = INV(data)
test.run(new_pp)
test.summary(3)
test.visual_summary()

Evaluating:   0%|          | 0/38 [00:00<?, ?it/s]

Predicting 297 examples


Evaluating: 100%|██████████| 38/38 [00:04<00:00,  9.20it/s]


Test cases:      27
Fails (rate):    4 (14.8%)

Example fails:
1.0 ("Who runs Trump 's Twitter account ? Does Donald Trump write his own tweets ?", 'Does Donald Trump really write his own tweets ?')
0.0 ("Who runs Trump 's Twitter account ? Does John Lopez write his own tweets ?", 'Does John Lopez really write his own tweets ?')
0.0 ("Who runs Trump 's Twitter account ? Does Joshua Thompson write his own tweets ?", 'Does Joshua Thompson really write his own tweets ?')

----
0.9 ('Did Hillary Clinton lie about Benghazi ?', 'Why did Hillary Clinton lie about Benghazi being attacked for a video instead of being honest to America ?')
0.0 ('Did Stephanie Ward lie about Benghazi ?', 'Why did Stephanie Ward lie about Benghazi being attacked for a video instead of being honest to America ?')
0.0 ('Did Nicole Martinez lie about Benghazi ?', 'Why did Nicole Martinez lie about Benghazi being attacked for a video instead of being honest to America ?')

----
1.0 ("What is Hillary Clinton 's policy 

TestSummarizer(stats={'nTested': 27, 'nFailed': 4, 'nFiltered': 0}, summarizer={'name': 'TESTNAME_PLACEHOLDER'…

In [6]:
test.visual_summary()

{'text': ('Will Donald Trump end up in some conflict of interest problem ?', 'Will Donald Trump end up in some conflict of interest ?'), 'pred': '1', 'conf': 0.9977355}
{'text': ('Michael Lee end up in some conflict of interest problem ?', 'Michael Lee end up in some conflict of interest ?'), 'pred': '1', 'conf': 0.996165}
{'text': ('Will Donald Trump end up in some conflict of interest problem ?', 'Will Donald Trump end up in some conflict of interest ?'), 'pred': '1', 'conf': 0.9977355, 'tokens': [['Will', 'Donald', 'Trump', 'end', 'up', 'in', 'some', 'conflict', 'of', 'interest', 'problem', '?'], ['Will', 'Donald', 'Trump', 'end', 'up', 'in', 'some', 'conflict', 'of', 'interest', '?']]}
{'text': ('Christopher Carter end up in some conflict of interest problem ?', 'Christopher Carter end up in some conflict of interest ?'), 'pred': '1', 'conf': 0.9965598}
{'text': ('What can we learn from Lord Krishna ?', "Where is Lord Krishna 's diamond ?"), 'pred': '0', 'conf': 0.9999566}
{'text':

TestSummarizer(stats={'nTested': 27, 'nFailed': 4, 'nFiltered': 0}, summarizer={'name': 'TESTNAME_PLACEHOLDER'…

# Vocabulary

In [5]:
professions = editor.suggest('{first_name} works as {a:bert}.')[:30]
print(', '.join(professions))

accountant, engineer, journalist, attorney, editor, artist, architect, actor, interpreter, analyst, intern, assistant, escort, organizer, economist, actress, administrator, investigator, agent, investor, secretary, author, entrepreneur, executive, advisor, auditor, educator, instructor, employee, adviser


In [20]:
#professions = editor.suggest('{first_name} works as {a:bert}.')[:30]
#professions += editor.suggest('{first_name} {last_name} works as {a:bert}.')[:30]
#professions = list(set(professions))
professions = []
# professions
other_nouns = ['player', 'person', 'friend', 'kid', 'candidate']
nouns = list(set(professions + other_nouns))

In [22]:
#print(', '.join(editor.suggest('Is {first_name} {last_name} {a:bert} {noun}?', noun=nouns)[:50]))
adjs = ['effective', 'actual', 'American', 'active', 'honest', 'excellent', 'elite', 'acomplished', 'official', 'outstanding', 'experienced', 'independent', 'international', 'aspiring', 'average', 'good', 'amazing', 'exceptional', 'successful', 'accredited', 'English', 'real', 'bad', 'terrible', 'fake', 'unusual', 'influential', 'incompetent']


In [23]:
data = editor.template(('Is {first_name} {last_name} {a:noun}?', 'Is {first_name} {last_name} {a:adj} {noun}?'),
                noun=nouns,
                adj=adjs,
                remove_duplicates=True, 
                nsamples=1000)
test = MFT(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 999 examples


Evaluating: 100%|██████████| 125/125 [00:13<00:00,  9.33it/s]

Test cases:      999
Fails (rate):    415 (41.5%)

Example fails:
0.7 ('Is Ryan Long a kid?', 'Is Ryan Long an effective kid?')
----
1.0 ('Is Danielle Price a friend?', 'Is Danielle Price an average friend?')
----
1.0 ('Is Amy King a player?', 'Is Amy King an average player?')
----


In [24]:
test.visual_summary()

TestSummarizer(stats={'nTested': 999, 'nFailed': 415, 'nFiltered': 0}, summarizer={'name': 'TESTNAME_PLACEHOLD…

# NER

## Change same name, number, location in both

In [7]:
test.visual_summary()

<class 'tuple'> ("Why did n't Michael Collins land on the moon ?", "Why did n't Michael Collins step on the Moon ?")
<class 'tuple'> ("Why did n't Michael Butler land on the moon ?", "Why did n't Michael Butler step on the Moon ?")
<class 'list'> [['Why', 'did', "n't", 'Michael', 'Collins', 'land', 'on', 'the', 'moon', '?'], ['Why', 'did', "n't", 'Michael', 'Collins', 'step', 'on', 'the', 'Moon', '?']]


TypeError: Argument 'string' has incorrect type (expected str, got list)

In [32]:
test.results.expect_results[1]

## Change name, loc, number in only one where orig prediction is duplicate

In [7]:
def wrap_apply_to_each(fn, both=False, *args, **kwargs):
    def new_fn(qs, *args, **kwargs):
        q1, q2 = qs
        ret = []
        fnq1 = fn(q1, *args, **kwargs)
        fnq2 = fn(q2, *args, **kwargs)
        if type(fnq1) != list:
            fnq1 = [fnq1]
        if type(fnq2) != list:
            fnq2 = [fnq2]
        ret.extend([(x, str(q2)) for x in fnq1])
        ret.extend([(str(q1), x) for x in fnq2])
        if both:
            ret.extend([(x, x2) for x, x2 in itertools.product(fnq1, fnq2)])
        return [x for x in ret if x[0] and x[1]]
    return new_fn
data = Perturb.perturb(parsed_qs, wrap_apply_to_each(Perturb.change_names), nsamples=1500, first_only=True)
expect_fn = Expect.eq(0)
expect_fn = Expect.slice_orig(expect_fn, lambda orig, *args: orig == 1)
test = DIR(data, expect_fn)
test.run(new_pp)
test.summary(3)


Predicting 1332 examples


Evaluating: 100%|██████████| 167/167 [00:17<00:00,  9.38it/s]

Test cases:      82
After filtering: 43 (52.4%)
Fails (rate):    39 (90.7%)

Example fails:
1.0 ('What do you think when Donald Trump is the president of the USA ?', 'Donald trump : what could happen if Donald Trump becomes president ?')
1.0 ('What do you think when Donald Trump is the president of the USA ?', 'Peter trump : what could happen if Peter Trump becomes president ?')
1.0 ('What do you think when Donald Trump is the president of the USA ?', 'Peter trump : what could happen if Peter Trump becomes president ?')

----
1.0 ('What would be the good things of a Trump presidency ?', 'What are some ways that a Donald Trump presidency may actually be good for America ?')
1.0 ('What would be the good things of a Trump presidency ?', 'What are some ways that a Jacob Trump presidency may actually be good for America ?')
1.0 ('What would be the good things of a Trump presidency ?', 'What are some ways that a Matthew Trump presidency may actually be good for America ?')

----
0.9 ('Am I t

In [8]:
test.visual_summary()

{'text': ('Is Quora supporting Hillary Clinton ?', 'Does Quora support Hillary Clinton ?'), 'pred': '1', 'conf': 0.9860616}
{'text': ('Is Quora supporting Hillary Clinton ?', 'Does Quora support Melissa Clinton ?'), 'pred': '1', 'conf': 0.99678963}
{'text': ('Is Quora supporting Hillary Clinton ?', 'Does Quora support Hillary Clinton ?'), 'pred': '1', 'conf': 0.9860616, 'tokens': [['Is', 'Quora', 'supporting', 'Hillary', 'Clinton', '?'], ['Does', 'Quora', 'support', 'Hillary', 'Clinton', '?']]}
{'text': ('Is Quora supporting Hillary Clinton ?', 'Does Quora support Samantha Clinton ?'), 'pred': '1', 'conf': 0.99580497}
{'text': ('Do I need an anti-virus for my mac ?', 'Does a Mac need anti-virus software ? Why and why not ?'), 'pred': '1', 'conf': 0.98453337}
{'text': ('Do I need an anti-virus for my mac ?', 'Does a Alexander need anti-virus software ? Why and why not ?'), 'pred': '0', 'conf': 0.9995154}
{'text': ('Do I need an anti-virus for my mac ?', 'Does a Mac need anti-virus softw

TestSummarizer(stats={'nTested': 43, 'nFailed': 39, 'nFiltered': 0}, summarizer={'name': 'TESTNAME_PLACEHOLDER…

In [35]:
test.results.preds[0]

array([1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0])

In [36]:
test.results.confs[0]

array([[6.2218524e-04, 9.9937773e-01],
       [9.8766202e-01, 1.2337957e-02],
       [1.5122294e-03, 9.9848771e-01],
       [1.8361963e-01, 8.1638026e-01],
       [9.8463297e-01, 1.5366935e-02],
       [8.3487844e-01, 1.6512157e-01],
       [2.0652901e-02, 9.7934705e-01],
       [9.7099119e-01, 2.9008875e-02],
       [9.4187868e-01, 5.8121365e-02],
       [3.8398802e-03, 9.9616027e-01],
       [9.9241483e-01, 7.5850831e-03]], dtype=float32)

In [34]:
test.results.expect_results[0]

array([-0.99937781,  1.        , -0.99848777, -0.81638037,  1.        ,
        1.        , -0.9793471 ,  1.        ,  1.        , -0.99616012,
        1.        ])

In [2]:

import checklist.viewer

In [3]:
print(checklist.viewer)

<module 'checklist.viewer' from '/home/wtshuang/sourcetree/checklist_ui/checklist/viewer/__init__.py'>


In [5]:
from checklist.viewer.viewer import TestSummarizer

In [7]:
summarize = {
            "tags": [],
            "name": "self.name",
            "result": {
                "nTested": 0,
                "nFailed": 0
            },
            "category": "Test"
        }

TestSummarizer(summarize, [])

TestSummarizer(stats={'nTested': 0, 'nFailed': 0}, summarizer={'tags': [], 'name': 'self.name', 'result': {'nT…

In [8]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [9]:
from viewer.viewer import TemplateEditor

TemplateEditor(
    sentences= {"Q1": "sentence1", "Q2": "sentence2"},
    suggest_fn=None,
    sources=[],
    allowed_sources=[],
    add_word_list_fn=None,
    get_testcase_fn=None,
    processor=nlp)

ModuleNotFoundError: No module named 'viewer'

In [2]:
def test():
    a=1
    b=2
    print(locals())

In [3]:
test()

{'a': 1, 'b': 2}


In [8]:
pos = ['good', 'bad', 'great']
def expect(x):
    return x in pos, locals()

In [9]:
expect("good")

(True, {'x': 'good'})

In [10]:
locals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'def test():\n    a=1\n    b=2\n    print(locals()',
  'def test():\n    a=1\n    b=2\n    print(locals())',
  'test()',
  "pos = ['good', 'bad', 'great']\ndef expect(x):\n    return x in pos, locals()",
  'expect("good")',
  "pos = ['good', 'bad', 'great']\ndef expect(x):\n    return x in pos, globals()",
  'expect("good")',
  "pos = ['good', 'bad', 'great']\ndef expect(x):\n    return x in pos, locals()",
  'expect("good")',
  'locals()'],
 '_oh': {5: (True, {'x': 'good'}), 7: (True, {...}), 9: (True, {'x': 'good'})},
 '_dh': ['/home/wtshuang/sourcetree/checklist_ui/notebooks'],
 'In': ['',
  'def test():\n    a=1\n    b=2\n    print(locals()',
  'def test():\n    a=1\n    b=2\n    print(locals())',
  'test()'

In [2]:
pos = ['good', 'bad', 'great']
def expect(x):
    return x in pos

In [4]:
import dill as pickle
pickle.dump(expect, open('./try_pickle.pkl', 'wb'), recurse=True)

In [3]:
import dill as pickle
expect = pickle.load(open('./try_pickle.pkl', 'rb'))

EOFError: Ran out of input